In [2]:
import pandas as pd

In [3]:
pelis = pd.read_csv('./top10K-TMDB-movies.csv')

In [4]:
pelis.head()

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811


In [5]:
pelis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10000 non-null  int64  
 1   title              10000 non-null  object 
 2   genre              9997 non-null   object 
 3   original_language  10000 non-null  object 
 4   overview           9987 non-null   object 
 5   popularity         10000 non-null  float64
 6   release_date       10000 non-null  object 
 7   vote_average       10000 non-null  float64
 8   vote_count         10000 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 703.3+ KB


In [6]:
pelis = pelis.dropna()
pelis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9985 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 9985 non-null   int64  
 1   title              9985 non-null   object 
 2   genre              9985 non-null   object 
 3   original_language  9985 non-null   object 
 4   overview           9985 non-null   object 
 5   popularity         9985 non-null   float64
 6   release_date       9985 non-null   object 
 7   vote_average       9985 non-null   float64
 8   vote_count         9985 non-null   int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 780.1+ KB


In [7]:
columnas =pelis.columns

In [8]:
print(columnas[2])

genre


In [9]:
pelis = pelis[[columnas[0],columnas[1],columnas[4],columnas[2] ]]

In [10]:
pelis['etiquetas'] = pelis['overview']+pelis['genre']

In [11]:
datosProcesados = pelis.drop(['overview','genre'],axis=1)

In [12]:
datosProcesados.head()

,id,title,etiquetas
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,In the continuing saga of the Corleone crime f...


# ENTRENO DEL MODELO

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
cv = CountVectorizer(max_features=9985, stop_words='english')

In [16]:
cv

CountVectorizer(max_features=9985, stop_words='english')

In [17]:
vector = cv.fit_transform(datosProcesados['etiquetas'].values.astype('U')).toarray()

In [18]:
vector.shape

(9985, 9985)

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
similarity = cosine_similarity(vector)

In [21]:
similarity

array([[1.        , 0.05634362, 0.13041013, ..., 0.07559289, 0.11065667,
        0.06900656],
       [0.05634362, 1.        , 0.07715167, ..., 0.        , 0.03636965,
        0.        ],
       [0.13041013, 0.07715167, 1.        , ..., 0.02300219, 0.0673435 ,
        0.09449112],
       ...,
       [0.07559289, 0.        , 0.02300219, ..., 1.        , 0.03253   ,
        0.03042903],
       [0.11065667, 0.03636965, 0.0673435 , ..., 0.03253   , 1.        ,
        0.04454354],
       [0.06900656, 0.        , 0.09449112, ..., 0.03042903, 0.04454354,
        1.        ]])

In [22]:
similarity.shape

(9985, 9985)

In [23]:
indexABuscar = datosProcesados[datosProcesados['title'] == "The Godfather: Part II"].index[0]

In [24]:
datosProcesados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9985 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         9985 non-null   int64 
 1   title      9985 non-null   object
 2   etiquetas  9985 non-null   object
dtypes: int64(1), object(2)
memory usage: 312.0+ KB


In [25]:
distancias = sorted(list(enumerate(similarity[indexABuscar])), reverse = True, key= lambda vector:vector[1])

In [26]:
for i in distancias[0:5]:
  print(i)
  print(datosProcesados.iloc[i[0]].title)


(4, 1.0)
The Godfather: Part II
(2, 0.4763305116224667)
The Godfather
(1611, 0.4021998332699218)
The Godfather: Part III
(9512, 0.31506301890630223)
Gotti
(7862, 0.291161615782696)
Mulholland Falls


# IMPLEMENTACIÓN DE PÁGINA WEB CON FLASK

In [28]:
!pip install flask flask-ngrok

In [29]:
!pip install pyngrok


In [30]:
from pyngrok import ngrok
#!ngrok config add-authtoken 2kzDHXELRRcFAebVUjUnay2qrZa_3jkkdFAEsAcsiQsM2GXup
# Abrir un túnel a la aplicación Flask en el puerto 5000
#public_url = ngrok.connect(5000)
#print("Ngrok public URL:", public_url)


In [31]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

#app = Flask(__name__)
#run_with_ngrok(app)

#@app.route('/get', methods=['GET'])
#def get():
 #   return jsonify({'response': 'Nigger'})


#if __name__ == '__main__':
    #app.run()

# HACER DEVELOPMENT WITH FLASK

In [33]:
def recomendar(movies):
  index = datosProcesados[datosProcesados['title'] == movies].index[0]
  distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1
                                                                                       ])
  recomendadas = []
  for i in distances[1:6]:
    recomendadas.append(datosProcesados.iloc[i[0]].title)
  return recomendadas

In [34]:
recomendar("Star Wars")

['The Empire Strikes Back',
 'Star Wars: The Force Awakens',
 'Return of the Jedi',
 'Star Wars: Episode III - Revenge of the Sith',
 'Star Trek VI: The Undiscovered Country']

In [35]:
import pickle

In [36]:
pickle.dump(datosProcesados, open('lista_peliculas.pkl', 'wb'))

In [37]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))